Counting Bananas in a complete banana bunch

In [ ]:
from fastai.vision import *
import pandas as pd

In [ ]:
# Load Labels
train_label = pd.read_csv("../input/banana-count-and-weight-in-a-bunch/Estu.csv", header=None)

In [ ]:
#if flip_vert=true (default), errors are too huge and do not decrease as we train.
tfms = get_transforms(flip_vert=False, max_lighting=0.2, max_zoom=1.05, max_warp=0.1)  

In [ ]:
data = (ImageList.from_csv(path='../input/banana-count-and-weight-in-a-bunch', csv_name='Estu.csv', folder='Banana_bunch_images')
        .split_by_rand_pct()
        .label_from_df(label_cls=FloatList) #label_cls=FloatList ensures that this is handled as 'Regression', not 'Classification'
        .transform(tfms, size=224)  #without the 'size', it results in cuda out of memory
        .databunch())

In [ ]:
# data.normalize(imagenet_stats)  #To do. How the train works without normalization

In [ ]:
data.show_batch(rows=3, figsize=(9,9))

In [ ]:
# learn = cnn_learner(data, models.resnet34) #Todo. Check if other models can be used
learn = cnn_learner(data, models.resnet50) #Todo. Check if other models can be used
learn.loss = MSELossFlat # For Regression MSE loss is the Loss function

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(30, 0.05)  


#ToDo
1. Trainining loss reduces slowly, but validation loss is unstable. Need to find out why.
2. Above statement was run twice (first time with lr 0.1, second time with 0.5). It was observed that there was a sudden drop in training loss the second time. But the training loss still went on reducing for each epoch. The third time it did not reduce, rather moving sideways.
3. Ensure that a single bunch's different images are not shared both in valid and train sets.
4. Try with BW images
5. Change the background and train
6. While testing check how the same image works in different zoom level? roataion?

In [ ]:
learn.data.valid_ds[0][0].shape

In [ ]:
learn.data.valid_ds[0][0]

In [ ]:
learn.data.valid_ds[0][1]

In [ ]:
learn.predict(learn.data.valid_ds[0][0])

In [ ]:
# Predict for a New Image
# img = open_image('banana/test/IMG_20190401_084251.jpg')
img = open_image('banana/train/IMG_20190401_081240.jpg')
img


In [ ]:
learn.predict(img) 

In [ ]:
img.shape  #It is observed that whenever the size is bigger, there is a high accuracy. Need to verify. Also, can this work with resnet32 as it was trained with sz 224.

Questions
1. Any other model to choose?
2. What about traning size?
3. What does it mean to run learn.fit_one_cycle repeatedly? is it same as high number of epochs?


In [ ]:
learn.export()

In [ ]:
learn = load_learner('banana')

In [ ]:
il = ImageList.from_folder("banana/test/63")

In [ ]:
for image in il:
    image.show()
    print(learn.predict(image))